In [1]:
#Install uninstalled modules, comment if it's already installed

#!pip install -U MetaTrader5 matplotlib numpy pandas pandas_ta scikit-learn

In [2]:
#Python version
!python --version

Python 3.8.12


In [3]:
#Import needed modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as ta

import datetime, os, pytz
import MetaTrader5 as mt5

import config

import sklearn
import pickle

In [6]:
#Trading Account Parameter
account = config.username   #Account number
password = config.password  #Password number
server = config.mt5_server  #Server name
path = config.mt5_path      #path of Metatrader5 director

mt5.initialize(
   path = path,          # path to the MetaTrader 5 terminal EXE file
   login = account,      # account number
   password = password,  # password
   server = server,      # server name as it is specified in the terminal
   #timeout = TIMEOUT,   # timeout
   portable = False      # portable mode
   )

# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__,"\n")

# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize failed, error code =",mt5.last_error())
    mt5.shutdown()
else:
    print("MetaTrader5 Initialized!")

# extract account information and turn it into dataframe   
authorized=mt5.login(account, password, server)
if authorized:
    # display trading account data 'as is'
    print("Metatrader5 authorized!")
    account_info_dict = mt5.account_info()._asdict()
    Acc_Info = pd.DataFrame(list(account_info_dict.items()),columns=['property','value'])
    print("\n", "account_info() as dataframe:")
    print(Acc_Info)
else:
    print("Metatrader5 cant authorize, error code: {}".format(mt5.last_error()))

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.36 

MetaTrader5 Initialized!
Metatrader5 cant authorize, error code: (-6, 'Terminal: Authorization failed')


In [7]:
#Extract Account info from dataframe
leverage = Acc_Info.loc[2, "value"]
equity = Acc_Info.loc[13, "value"]
margin_free = Acc_Info.loc[15, "value"]

print(leverage)
print(equity)
print(margin_free)

NameError: name 'Acc_Info' is not defined

In [ ]:
# extract information from pair and timeframe
Symbol = "EURUSD"
Timeframe = "TIMEFRAME_D1"
symbol_info = mt5.symbol_info(Symbol)
if symbol_info!=None:
    # display the terminal data 'as is'    
    print(symbol_info)
    print("\n","{}: spread =".format(Symbol),symbol_info.spread,"  digits =",symbol_info.digits, "\n")
    # Extract symbol info
    
    print(symbol_info[3])

SymbolInfo(custom=False, chart_mode=0, select=True, visible=True, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=0, volumehigh=0, volumelow=0, time=1624497435, digits=5, spread=10, spread_float=True, ticks_bookdepth=0, trade_calc_mode=0, trade_mode=4, start_time=0, expiration_time=0, trade_stops_level=1, trade_freeze_level=0, trade_exemode=2, swap_mode=1, swap_rollover3days=3, margin_hedged_use_leg=False, expiration_mode=15, filling_mode=3, order_mode=127, order_gtc_mode=0, ...)

 EURUSD: spread = 10   digits = 5 

True


In [ ]:
# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")

# getbars from the current day
bar = 500
rates = mt5.copy_rates_from_pos(Symbol, Timeframe, 0, bar)
 
# create DataFrame out of the obtained data
rates_frame = pd.DataFrame(rates, dtype=np.dtype("float"))
# convert time in seconds into the datetime format
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
                           
# display data
print("\nDisplay dataframe with data")
print(rates_frame)

# turn rates frame into csv file
rates_frame.to_csv('./Datasets/{}.csv'.format(Symbol), index=False)
pd.read_csv('./Datasets/{}.csv'.format(Symbol))


Display dataframe with data
          time     open     high      low    close  tick_volume  spread  \
0   2019-07-22  1.12144  1.12244  1.12052  1.12080      48433.0     8.0   
1   2019-07-23  1.12080  1.12087  1.11450  1.11507      50464.0     8.0   
2   2019-07-24  1.11508  1.11550  1.11261  1.11391      51590.0     8.0   
3   2019-07-25  1.11392  1.11869  1.11006  1.11455      64621.0     8.0   
4   2019-07-26  1.11455  1.11499  1.11112  1.11229      52303.0     8.0   
..         ...      ...      ...      ...      ...          ...     ...   
495 2021-06-18  1.19034  1.19212  1.18436  1.18584      63688.0     4.0   
496 2021-06-21  1.18541  1.19197  1.18459  1.19099      76173.0     4.0   
497 2021-06-22  1.19134  1.19508  1.18791  1.19315      63425.0     4.0   
498 2021-06-23  1.19342  1.19683  1.19096  1.19200      59965.0     4.0   
499 2021-06-24  1.19231  1.19307  1.19225  1.19296        211.0     0.0   

     real_volume  
0            0.0  
1            0.0  
2            

,time,open,high,low,close,tick_volume,spread,real_volume
0,2019-07-22,1.12144,1.12244,1.12052,1.12080,48433.0,8.0,0.0
1,2019-07-23,1.12080,1.12087,1.11450,1.11507,50464.0,8.0,0.0
2,2019-07-24,1.11508,1.11550,1.11261,1.11391,51590.0,8.0,0.0
3,2019-07-25,1.11392,1.11869,1.11006,1.11455,64621.0,8.0,0.0
4,2019-07-26,1.11455,1.11499,1.11112,1.11229,52303.0,8.0,0.0
...,...,...,...,...,...,...,...,...
495,2021-06-18,1.19034,1.19212,1.18436,1.18584,63688.0,4.0,0.0
496,2021-06-21,1.18541,1.19197,1.18459,1.19099,76173.0,4.0,0.0
497,2021-06-22,1.19134,1.19508,1.18791,1.19315,63425.0,4.0,0.0
498,2021-06-23,1.19342,1.19683,1.19096,1.19200,59965.0,4.0,0.0


In [ ]:
# read csv from datasets folder
df = pd.read_csv('./Datasets/{}.csv'.format(Symbol))

# turn time coloumn into datetime
df['time'] = pd.to_datetime(df['time'])

#delete empty coloumn
del df['real_volume']

# make indicator coloumns to the dataframe
df['EMA60'] = TA.EMA(df, 60, 'open')
df['EMA120'] = TA.EMA(df, 120, 'open')
df['EMA240'] = TA.EMA(df, 240, 'open')
df['RSI'] = TA.RSI(df, 14, 'open')
df['ATR'] = TA.ATR(df, 14)
df.fillna(0, inplace=True)

#  rename coloumn's name
df = df.rename(columns={
    "time":"Date",
    "open":"Open",
    "high":"High",
    "low":"Low",
    "close":"Close",
    "tick_volume":"Tick_volume",
    "spread":"Spread",
})

# set Date coloumn as index
df.set_index('Date', inplace=True)
df

,Open,High,Low,Close,Tick_volume,Spread,EMA60,EMA120,EMA240,RSI,ATR
Date,,,,,,,,,,,
2019-07-22,1.12144,1.12244,1.12052,1.12080,48433.0,8.0,1.121440,1.121440,1.121440,0.000000,0.000000
2019-07-23,1.12080,1.12087,1.11450,1.11507,50464.0,8.0,1.121115,1.121117,1.121119,0.000000,0.000000
2019-07-24,1.11508,1.11550,1.11261,1.11391,51590.0,8.0,1.119036,1.119071,1.119089,0.000000,0.000000
2019-07-25,1.11392,1.11869,1.11006,1.11455,64621.0,8.0,1.117692,1.117751,1.117781,0.000000,0.000000
2019-07-26,1.11455,1.11499,1.11112,1.11229,52303.0,8.0,1.117021,1.117089,1.117124,8.809213,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2021-06-18,1.19034,1.19212,1.18436,1.18584,63688.0,4.0,1.208767,1.203386,1.189825,30.110550,0.007124
2021-06-21,1.18541,1.19197,1.18459,1.19099,76173.0,4.0,1.208002,1.203089,1.189788,27.582423,0.007345
2021-06-22,1.19134,1.19508,1.18791,1.19315,63425.0,4.0,1.207455,1.202895,1.189801,34.686026,0.007399
